In [1]:
import pandas as pd 
from simpledbf import Dbf5
pd.options.display.float_format = '{:,.5f}'.format

In [2]:

dbf = Dbf5('1997dados.dbf')
df1997 = dbf.to_dataframe()
dbf = Dbf5('2007dados.dbf')
df2007 = dbf.to_dataframe()
dbf = Dbf5('2017dados.dbf')
df2017 = dbf.to_dataframe()


# FILTRA AS COLUNAS

In [6]:
df2017.columns

Index(['ZONA', 'MUNI_DOM', 'CO_DOM_X', 'CO_DOM_Y', 'ID_DOM', 'F_DOM', 'FE_DOM',
       'DOM', 'CD_ENTRE', 'DATA',
       ...
       'DURACAO', 'MODOPRIN', 'TIPVG', 'PAG_VIAG', 'TP_ESAUTO', 'VL_EST',
       'PE_BICI', 'VIA_BICI', 'TP_ESTBICI', 'ID_ORDEM'],
      dtype='object', length=126)

In [3]:
import utm
import math
import geopy.distance
def calculate_distance(row,origem, destino):  
    return geopy.distance.geodesic(row[origem],row[destino]).km

def convert_lat_long(row, column_x, column_y):  
    if math.isnan(row[column_x]) or math.isnan(row[column_y]):
        return 0
    else:
        return utm.to_latlon(row[column_x],row[column_y], 23, 'K')

df2007["origem"] = df2007.apply(lambda x: convert_lat_long(x,'CO_DOM_X', 'CO_DOM_Y'), axis=1)
df2007["destino"] = df2007.apply(lambda x: convert_lat_long(x,'CO_D_X', 'CO_D_Y'), axis=1)
df2007["dist_ori_des"] = df2007[['origem','destino']].apply(lambda x: calculate_distance(x,'origem', 'destino'), axis=1)
df2017["origem"] = df2017.apply(lambda x: convert_lat_long(x,'CO_DOM_X', 'CO_DOM_Y'), axis=1)
df2017["destino"] = df2017.apply(lambda x: convert_lat_long(x,'CO_D_X', 'CO_D_Y'), axis=1)
df2017["dist_ori_des"] = df2017[['origem','destino']].apply(lambda x: calculate_distance(x,'origem', 'destino'), axis=1)


In [4]:
lista_2017 = ['ID_FAM','ID_PESS','FE_FAM','ZONA','VL_REN_I','CRITERIOBR','RENDA_FA','ZONA_O','ZONA_D','MOTIVO_O','MOTIVO_D','MODOPRIN','DURACAO','TIPVG','FE_VIA','dist_ori_des']
lista_2007 = ['ID_FAM','ID_PESS','FE_FAM','ZONA','VL_REN_I','CRITERIO_B','RENDA_FA','ZONA_O','MUNI_O','ZONA_D','MOTIVO_O','MOTIVO_D','MODOPRIN','DURACAO','TIPOVG','FE_VIA','dist_ori_des']
lista_1997 = ['ID_FAM','ID_PESS','FE_FAM','ZONA','VL_REN_I','RENDA_FA','RENDATRI','ABIPEME','ZONA_O','ZONA_D','MOTIVO_O','MOTIVO_D','MODOPRIN','DURACAO','TIPO_VG','FE_VIA']

df97 = df1997[lista_1997].copy()
df97 = df97[df97['DURACAO']!=0]
df97.columns = map(str.lower, df97.columns)

df07 = df2007[lista_2007].copy()

df07 = df07[df07['DURACAO']!=0]
df07.columns = map(str.lower, df07.columns)

df17 = df2017[lista_2017].copy()
df17 = df17[df17['DURACAO']!=0]
df17.columns = map(str.lower, df17.columns)

# ADICIONAR NOME DE ZONAS

In [5]:

df_zonas17 = pd.read_excel("layout_todos.xlsx", sheet_name = 'divisao2017')
corr_1707 = pd.read_excel("layout_todos.xlsx", sheet_name = 'zona2017-2007')[['numero_zona2007','numero_zona2017']]

corr_1797 = pd.read_excel("layout_todos.xlsx", sheet_name = 'zona2007-1997')[['numero_zona2007','numero_zona1997']]\
.merge(corr_1707[['numero_zona2007','numero_zona2017']], on = 'numero_zona2007',how  = 'inner')




# zona97, zona07->zona 2017

In [6]:
df97 = pd.merge(df97,corr_1797[['numero_zona1997','numero_zona2017']],left_on=['zona'], right_on=['numero_zona1997'],how = 'inner')
df97 = df97[[i for i in list(df97.columns) if i not in ['zona','numero_zona1997']]].rename(columns = {'numero_zona2017':'zona_dom'})
df97 = pd.merge(df97,corr_1797[['numero_zona1997','numero_zona2017']],left_on=['zona_o'], right_on=['numero_zona1997'],how = 'inner')
df97 = df97[[i for i in list(df97.columns) if i not in ['zona_o','numero_zona1997']]].rename(columns = {'numero_zona2017':'zona_origem'})
df97 = pd.merge(df97,corr_1797[['numero_zona1997','numero_zona2017']],left_on=['zona_d'], right_on=['numero_zona1997'],how = 'inner')
df97 = df97[[i for i in list(df97.columns) if i not in ['zona_d','numero_zona1997']]].rename(columns = {'numero_zona2017':'zona_destino'})

df07 = pd.merge(df07,corr_1707[['numero_zona2007','numero_zona2017']],left_on=['zona'], right_on=['numero_zona2007'],how = 'inner')
df07 = df07[[i for i in list(df07.columns) if i not in ['zona','numero_zona2007']]].rename(columns = {'numero_zona2017':'zona_dom'})
df07 = pd.merge(df07,corr_1707[['numero_zona2007','numero_zona2017']],left_on=['zona_o'], right_on=['numero_zona2007'],how = 'inner')
df07 = df07[[i for i in list(df07.columns) if i not in ['zona_o','numero_zona2007']]].rename(columns = {'numero_zona2017':'zona_origem'})
df07 = pd.merge(df07,corr_1707[['numero_zona2007','numero_zona2017']],left_on=['zona_d'], right_on=['numero_zona2007'],how = 'inner')
df07 = df07[[i for i in list(df07.columns) if i not in ['zona_d','numero_zona2007']]].rename(columns = {'numero_zona2017':'zona_destino'})


df07 = df07.rename(columns = {'zona_dom':'numero_zonadom','zona_origem':'zona_o','zona_destino':'zona_d'})
df97 = df97.rename(columns = {'zona_dom':'numero_zonadom','zona_origem':'zona_o','zona_destino':'zona_d'})


# zona_name

In [7]:
df17 = pd.merge(df17,df_zonas17[['numero_zona','zona']],left_on=['zona'], right_on=['numero_zona'],how = 'inner').rename(columns = {'zona_y':'zona_dom'})
df17 = df17[[i for i in list(df17.columns) if i not in ['zona_x']]]
df17 = df17.rename(columns = {'numero_zona':'numero_zonadom'})
df17 = pd.merge(df17,df_zonas17[['numero_zona','zona']],left_on=['zona_o'], right_on=['numero_zona'],how = 'inner').rename(columns = {'zona':'zona_origem'})
df17 = df17[[i for i in list(df17.columns) if 'numero_zona_' not in i ]]
df17 = pd.merge(df17,df_zonas17[['numero_zona','zona']],left_on=['zona_d'], right_on=['numero_zona'],how = 'inner').rename(columns = {'zona':'zona_destino'})
df17 = df17[[i for i in list(df17.columns)  if 'numero_zona_' not in i]]

In [323]:
df07.columns

Index(['id_fam', 'id_pess', 'fe_fam', 'vl_ren_i', 'criterio_b', 'renda_fa',
       'muni_o', 'motivo_o', 'motivo_d', 'modoprin', 'duracao', 'tipovg',
       'fe_via', 'dist_ori_des', 'numero_zonadom', 'zona_o', 'zona_d',
       'zona_dom', 'zona_origem', 'zona_destino'],
      dtype='object')

In [8]:
df07 = pd.merge(df07,df_zonas17[['numero_zona','zona']],left_on=['numero_zonadom'], right_on=['numero_zona'],how = 'inner')
df07 = df07.rename(columns = {'zona':'zona_dom'})
df07 = df07[[i for i in list(df07.columns) if i not in ['numero_zona']]]
df07 = pd.merge(df07,df_zonas17[['numero_zona','zona']],left_on=['zona_o'], right_on=['numero_zona'],how = 'inner')
df07 = df07.rename(columns = {'zona':'zona_origem'})
df07 = df07[[i for i in list(df07.columns) if i not in ['numero_zona']]]
df07 = pd.merge(df07,df_zonas17[['numero_zona','zona']],left_on=['zona_d'], right_on=['numero_zona'],how = 'inner')
df07 = df07.rename(columns = {'zona':'zona_destino'})
df07 = df07[[i for i in list(df07.columns) if i not in ['numero_zona']]]

In [9]:
df97 = pd.merge(df97,df_zonas17[['numero_zona','zona']],left_on=['numero_zonadom'], right_on=['numero_zona'],how = 'inner')
df97 = df97.rename(columns = {'zona':'zona_dom'})
df97 = df97[[i for i in list(df97.columns) if i not in ['numero_zona']]]
df97 = pd.merge(df97,df_zonas17[['numero_zona','zona']],left_on=['zona_o'], right_on=['numero_zona'],how = 'inner')
df97 = df97.rename(columns = {'zona':'zona_origem'})
df97 = df97[[i for i in list(df97.columns) if i not in ['numero_zona']]]
df97 = pd.merge(df97,df_zonas17[['numero_zona','zona']],left_on=['zona_d'], right_on=['numero_zona'],how = 'inner')
df97 = df97.rename(columns = {'zona':'zona_destino'})
df97 = df97[[i for i in list(df97.columns) if i not in ['numero_zona']]]

# add distrito

In [10]:
df17 = pd.merge(df17,df_zonas17[['numero_zona','numero_distrito','distrito']],left_on=['numero_zonadom'], right_on=['numero_zona'],how = 'inner')\
.rename(columns = {'distrito':'distrito_dom','numero_distrito':'numero_distrito_dom'})
df17 = df17[[i for i in list(df17.columns) if i not in ['zona_x']]]
df17 = pd.merge(df17,df_zonas17[['numero_zona','numero_distrito','distrito']],left_on=['zona_o'], right_on=['numero_zona'],how = 'inner')\
.rename(columns = {'distrito':'distrito_origem','numero_distrito':'numero_distrito_origem'})
df17 = df17[[i for i in list(df17.columns) if i not in ['numero_zona']]]
df17 = pd.merge(df17,df_zonas17[['numero_zona','numero_distrito','distrito']],left_on=['zona_d'], right_on=['numero_zona'],how = 'inner')\
.rename(columns = {'distrito':'distrito_destino','numero_distrito':'numero_distrito_destino'})
df17 = df17[[i for i in list(df17.columns)   if i not in ['numero_zona']]]

df97 = pd.merge(df97,df_zonas17[['numero_zona','numero_distrito','distrito']],left_on=['numero_zonadom'], right_on=['numero_zona'],how = 'inner')\
.rename(columns = {'distrito':'distrito_dom','numero_distrito':'numero_distrito_dom'})
df97 = df97[[i for i in list(df97.columns) if i not in ['zona_x']]]
df97 = pd.merge(df97,df_zonas17[['numero_zona','numero_distrito','distrito']],left_on=['zona_o'], right_on=['numero_zona'],how = 'inner')\
.rename(columns = {'distrito':'distrito_origem','numero_distrito':'numero_distrito_origem'})
df97 = df97[[i for i in list(df97.columns) if i not in ['numero_zona']]]
df97 = pd.merge(df97,df_zonas17[['numero_zona','numero_distrito','distrito']],left_on=['zona_d'], right_on=['numero_zona'],how = 'inner')\
.rename(columns = {'distrito':'distrito_destino','numero_distrito':'numero_distrito_destino'})
df97 = df97[[i for i in list(df97.columns)  if i not in ['numero_zona']]]

df07 = pd.merge(df07,df_zonas17[['numero_zona','numero_distrito','distrito']],left_on=['numero_zonadom'], right_on=['numero_zona'],how = 'inner')\
.rename(columns = {'distrito':'distrito_dom','numero_distrito':'numero_distrito_dom'})
df07 = df07[[i for i in list(df07.columns) if i not in ['zona_x']]]
df07 = pd.merge(df07,df_zonas17[['numero_zona','numero_distrito','distrito']],left_on=['zona_o'], right_on=['numero_zona'],how = 'inner')\
.rename(columns = {'distrito':'distrito_origem','numero_distrito':'numero_distrito_origem'})
df07 = df07[[i for i in list(df07.columns) if i not in ['numero_zona']]]
df07 = pd.merge(df07,df_zonas17[['numero_zona','numero_distrito','distrito']],left_on=['zona_d'], right_on=['numero_zona'],how = 'inner')\
.rename(columns = {'distrito':'distrito_destino','numero_distrito':'numero_distrito_destino'})
df07 = df07[[i for i in list(df07.columns)   if i not in ['numero_zona']]]

In [11]:
df17 = df17[[i for i in list(df17.columns)   if 'numero_zona_' not in i]]
df07 = df07[[i for i in list(df07.columns)   if 'numero_zona_' not in i]]
df97 = df97[[i for i in list(df97.columns)   if 'numero_zona_' not in i]]


In [326]:
df97.columns

Index(['id_fam', 'id_pess', 'fe_fam', 'vl_ren_i', 'renda_fa', 'rendatri',
       'abipeme', 'motivo_o', 'motivo_d', 'modoprin', 'duracao', 'tipo_vg',
       'fe_via', 'numero_zonadom', 'zona_o', 'zona_d', 'zona_dom',
       'zona_origem', 'zona_destino', 'numero_distrito_dom', 'distrito_dom',
       'numero_distrito_origem', 'distrito_origem', 'numero_distrito_destino',
       'distrito_destino'],
      dtype='object')

# add regiao

In [12]:
import numpy as np
def add_regiao(df, nome_column):
    conditions = [
    (df[nome_column] <109) ,
    ((df[nome_column] >= 109) & (df[nome_column] <128)),
    (df[nome_column]>=128)& (df[nome_column] <166),
    (df[nome_column]>=166)& (df[nome_column] <208),
    (df[nome_column]>=208)& (df[nome_column] <236),
    (df[nome_column]>=236)& (df[nome_column] <268),
    (df[nome_column]>=268)& (df[nome_column] <300),
    (df[nome_column]>=300)& (df[nome_column] <316),
    (df[nome_column]>=316)& (df[nome_column] <343)]
    choices = ['central', 'noroeste', 'norte','nordeste','leste','sudeste','sul','sudoeste','oeste']   
    df['regiao_central_{}'.format(nome_column.split('_')[-1])] = np.select(conditions, choices)
    conditions = [
    (df[nome_column] <343) ,
    ((df[nome_column] >= 343) & (df[nome_column] <362)),
    (df[nome_column]>=362)& (df[nome_column] <391),
    (df[nome_column]>=391)& (df[nome_column] <417),
    (df[nome_column]>=417)& (df[nome_column] <460),
    (df[nome_column]>=460)& (df[nome_column] <476),
    (df[nome_column]>=476)& (df[nome_column] <518)]
    choices = ['central', 'norte', 'nordeste','leste','sudeste','sudoeste','oeste']
    df['regiao_{}'.format(nome_column.split('_')[-1])] = np.select(conditions, choices)
    return df

df07 = add_regiao(df07, 'numero_zonadom')
df07 = add_regiao(df07, 'zona_o')
df07 = add_regiao(df07, 'zona_d')

df97 = add_regiao(df97, 'numero_zonadom')
df97 = add_regiao(df97, 'zona_o')
df97 = add_regiao(df97, 'zona_d')

df17 = add_regiao(df17, 'numero_zonadom')
df17 = add_regiao(df17, 'zona_o')
df17 = add_regiao(df17, 'zona_d')
df17 = df17.rename(columns = {'regiao_central_zonadom':'regiao_central_dom','regiao_central_o':'regiao_central_origem',
                              'regiao_zonadom':'regiao_dom','regiao_o':'regiao_origem','regiao_central_d':'regiao_central_destino',
                             'regiao_d':'regiao_destino'})
df07 = df07.rename(columns = {'regiao_central_zonadom':'regiao_central_dom','regiao_central_o':'regiao_central_origem',
                              'regiao_zonadom':'regiao_dom','regiao_o':'regiao_origem','regiao_central_d':'regiao_central_destino',
                             'regiao_d':'regiao_destino'})
df97 = df97.rename(columns = {'regiao_central_zonadom':'regiao_central_dom','regiao_central_o':'regiao_central_origem',
                              'regiao_zonadom':'regiao_dom','regiao_o':'regiao_origem','regiao_central_d':'regiao_central_destino',
                             'regiao_d':'regiao_destino'})

In [13]:


df17['count'] = 1
df07['count'] = 1
df97['count'] = 1


# REPLACE NOME DE ITENS POR NUMERO DE ITEMS

# 2017

In [14]:
dict_motivo17 = {1:"Trabalho",
2:"Trabalho",
3:"Trabalho",
4:"Escola/Educação",
5:"Compras",
6:"Médico/Dentista/Saúde",
7:"Recreação/Visitas/Lazer",
8:"Residência",
9:"Procurar Emprego",
10:"Assuntos Pessoais",
11:"Refeição"}
df17['motivo_na_origem_nome'] = df17['motivo_o'].replace(dict_motivo17)
df17['motivo_no_destino_nome'] = df17['motivo_d'].replace(dict_motivo17)
df17 = df17[[i for i in (df17.columns) if i not in ['motivo_o','motivo_d']]]

dict_criteriobr17 = {
1:"A",
2:"B",
3:"B",
4:"C",
5:"C",
6:"D - E"
}


df17['criteriobr'] = df17['criteriobr'].replace(dict_criteriobr17)


dict_modos17 = {
1:"Metrô",
2:"Trem",
3:"Monotrilho",
4:"Ônibus",
5:"Ônibus",
6:"Ônibus",
7:"Fretado",
8:"Escolar",
9:"Carro",
10:"Táxi/Uber",
11:"Táxi/Uber",
12:"Táxi/Uber",
13:"Moto",
14:"Moto",
15:"Bicicleta",
16:"A Pé",
17:"Outros"
}



df17['modoprin'] = df17['modoprin'].replace(dict_modos17)

tipoviagem17 = {
1:"Coletivo",
2:"Individual",
3:"A pé",
4:"Bicicleta"}

df17['tipvg'] = df17['tipvg'].replace(tipoviagem17)
#df17.to_csv('df17.csv')
##########################################################################################################################
dfresistencia17 = df17[(df17['motivo_no_destino_nome'].isin(['Residência'])) &(df17['motivo_na_origem_nome'].isin(["Trabalho"]))]

dftrabalho17 = df17[(df17['motivo_no_destino_nome'].isin(["Trabalho"])) &(df17['motivo_na_origem_nome'].isin(['Residência']))]


# 2007

In [15]:
dict_motivo07 = {1:"Trabalho",
2:"Trabalho",
3:"Trabalho",
4:"Escola/Educação",
5:"Compras",
6:"Médico/Dentista/Saúde",
7:"Recreação/Visitas/Lazer",
8:"Residência",
9:"Procurar Emprego",
10:"Assuntos Pessoais"}
df07['motivo_na_origem_nome'] = df07['motivo_o'].replace(dict_motivo07)
df07['motivo_no_destino_nome'] = df07['motivo_d'].replace(dict_motivo07)
df07 = df07[[i for i in (df07.columns) if i not in ['motivo_o','motivo_d']]]

dict_criteriobr07 = {1:"A",
2:"A",
3:"B",
4:"B",
5:"C",
6:"C",
7:"D - E",
8:"D - E"}


df07['criterio_b'] = df07['criterio_b'].replace(dict_criteriobr07)


dict_modos07 = {1:"Ônibus",
2:"Ônibus",
3:"Ônibus",
4:"Fretado",
5:"Escolar",
6:"Carro",
7:"Táxi/Uber",
8:"Táxi/Uber",
9:"Ônibus",
10:"Ônibus",
11:"Ônibus",
12:"Metrô",
13:"Trem",
14:"Moto",
15:"Bicicleta",
16:"A Pé",
17:"Outros"}

df07['modoprin'] = df07['modoprin'].replace(dict_modos07)

tipoviagem07 = {
1:"Coletivo",
2:"Individual",
3:"A pé",
4:'Bicicleta'}

df07['tipovg'] = df07['tipovg'].replace(tipoviagem07)

#df07.to_csv('df07.csv')
####################################################################################################################

dfresistencia07 = df07[(df07['motivo_no_destino_nome'].isin(['Residência'])) &(df07['motivo_na_origem_nome'].isin(["Trabalho"]))]

dftrabalho07 = df07[(df07['motivo_no_destino_nome'].isin(["Trabalho"])) &(df07['motivo_na_origem_nome'].isin(['Residência']))]

# 1997

In [16]:
dict_motivo97 = {1:"Trabalho",
2:"Trabalho",
3:"Trabalho",
4:"Escola/Educação",
5:"Compras",
6:"Médico/Dentista/Saúde",
7:"Recreação/Visitas/Lazer",
8:"Residência",
9:"outros"}
df97['motivo_na_origem_nome'] = df97['motivo_o'].replace(dict_motivo97)
df97['motivo_no_destino_nome'] = df97['motivo_d'].replace(dict_motivo97)
df97 = df97[[i for i in (df97.columns) if i not in ['motivo_o','motivo_d']]]

dict_criteriobr97 = {0:"A",
    1:"A",
2:"B",
3:"C",
4:"D - E",
5:"D - E"}


df97['abipeme'] = df97['abipeme'].replace(dict_criteriobr97)


dict_modos97 = {
1:"Ônibus",
2:"Fretado",
3:"Escolar",
4:"Carro",
5:"Táxi/Uber",
6:"Táxi/Uber",
7:"Ônibus",
8:"Metrô",
9:"Trem",
10:"Moto",
11:"Bicicleta",
12:"A Pé",
13:"Outros"
}

df97['modoprin'] = df97['modoprin'].replace(dict_modos97)
dfbici = df97[df97['modoprin'] =='Bicicleta'].copy()
dfbici['tipo_vg']=4
df972 = dfbici.append(df97[df97['modoprin'] !='Bicicleta'].copy(), ignore_index = True)
df97 = df972
tipoviagem97 = {
1:"Coletivo",
2:"Individual",
3:"A pé",
4:'Bicicleta'}

df97['tipo_vg'] = df97['tipo_vg'].replace(tipoviagem97)

#df97.to_csv('df97.csv')

#####################################################################################

dfresistencia97 = df97[(df97['motivo_no_destino_nome'].isin(['Residência'])) &(df97['motivo_na_origem_nome'].isin(["Trabalho"]))]

dftrabalho97 = df97[(df97['motivo_no_destino_nome'].isin(["Trabalho"])) &(df97['motivo_na_origem_nome'].isin(['Residência']))]


In [17]:
dftrabalho17 = dftrabalho17.drop_duplicates()
dftrabalho07 = dftrabalho07.drop_duplicates()
dftrabalho97 = dftrabalho97.drop_duplicates()
dfresistencia17 = dfresistencia17.drop_duplicates()
dfresistencia07 = dfresistencia07.drop_duplicates()
dfresistencia97 = dfresistencia97.drop_duplicates()


In [18]:
dftrabalho07 = dftrabalho07.rename(columns = {'criterio_b':'criteriobr','tipovg':'tipvg'})
dftrabalho97 = dftrabalho97.rename(columns = {'abipeme':'criteriobr','tipo_vg':'tipvg'})
dfresistencia07 = dfresistencia07.rename(columns = {'criterio_b':'criteriobr','tipovg':'tipvg'})
dfresistencia97 = dfresistencia97.rename(columns = {'abipeme':'criteriobr','tipo_vg':'tipvg'})
df07 = df07.rename(columns = {'criterio_b':'criteriobr','tipovg':'tipvg'})
df97 = df97.rename(columns = {'abipeme':'criteriobr','tipo_vg':'tipvg'})



In [20]:
df17.drop_duplicates().to_csv('df17.csv')
df07.drop_duplicates().to_csv('df07.csv')
df97.drop_duplicates().to_csv('df97.csv')

In [21]:
dftrabalho17.drop_duplicates().to_csv('dftrabalho17.csv')
dftrabalho07.drop_duplicates().to_csv('dftrabalho07.csv')
dftrabalho97.drop_duplicates().to_csv('dftrabalho97.csv')
dfresistencia17.drop_duplicates().to_csv('dfresistencia17.csv')
dfresistencia07.drop_duplicates().to_csv('dfresistencia07.csv')
dfresistencia97.drop_duplicates().to_csv('dfresistencia97.csv')
